#  Shuttle Search 

In [1]:
const input = fs
  .readFileSync("./input.txt", (_, a) => a)
  .toString()
  .split("\r\n")
  .map(a=> a);


# Part 1

What is the ID of the earliest shuttle you can take to the airport multiplied by the number of minutes you'll need to wait for that bus?

## Approach & Solution
Earliest time you can take a shuttle at an integer multiple of that shuttle's ID after you have arrived. Which translates to the ceiling of the division of the arrival time by shuttle's Id.
Or in mathamatical terms : 
###  $ t_{Shuttle} = ceil( \frac {Time_{arrival}} {Frequency_{Shuttle}}) = ceil(\frac {Time_{arr}} {ShuttleID} )$


In [2]:
const arrival = parseInt(input[0]);
const shuttleIds = input[1].split(',').filter(a =>a !='x').map(a=>parseInt(a));

const timeDiff = shuttleIds.map( a => {
    const match = Math.ceil(arrival/a); // Need Integer, Cannot take a shuttle that has already left; Hence Ceiling of the division
        return {
            id:a,
            timeDiff: (match*a)-arrival
        }
    }
).sort((a,b)=> a.timeDiff-b.timeDiff);

timeDiff[0].id * timeDiff[0].timeDiff;

3997

# Part 2
Use line two of input (shuttle ID) only. Find a time such that if the first shuttle leaves at time t= 0. The following shuttles leave at t+1, t+2, t+3... t+n. A shuttle X can leave at any time. 

## Approach
Compute the time delay $t_i$ required for each shuttle $s_i$ after the first one. The first one leaves at time t=0. $\therefore$ $t_0 = 0$


In [3]:
var shuttles = input[1].split(",");
var subsequentShuttles = [];

var currentShuttle = null;
var currentTimeDiff = 0;

for (let shuttle of shuttles) {
    if (!isNaN(shuttle)) {
        currentShuttle = parseInt(shuttle);
        subsequentShuttles.push({
            shuttle: parseInt(currentShuttle),
            delay: currentTimeDiff,
        });
    }
    currentTimeDiff += 1;
}

subsequentShuttles

[
  { shuttle: 19, delay: 0 },
  { shuttle: 41, delay: 9 },
  { shuttle: 37, delay: 13 },
  { shuttle: 787, delay: 19 },
  { shuttle: 13, delay: 32 },
  { shuttle: 23, delay: 42 },
  { shuttle: 29, delay: 48 },
  { shuttle: 571, delay: 50 },
  { shuttle: 17, delay: 67 }
]


The problem now is 

For $K$ shuttles we have to find $t$ $\backepsilon$ 

$ \quad (t + t_0) \mod s_0   \\
=   (t + t_1) \mod s_1  \\
 =   (t + t_2) \mod s_2  \\
  \vdots \\
  = (t + t_k) \mod s_k \\
  = 0
  $

Which is the equivalant to saying 

$ t \equiv -t_0\;(mod\;s_0) \equiv (s_0-t_0)\:(mod\;s_0) \\
  \: \equiv -t_1\;(mod\;s_1) \equiv (s_1-t_1)\:(mod\;s_1) \\
  \: \equiv -t_2\;(mod\;s_2) \equiv (s_2-t_2)\:(mod\;s_2) \\
  \: \vdots \\
  \: \equiv -t_k\;(mod\;s_k) \equiv (s_k-t_k)\:(mod\;s_k) \\
 $

Modify the subsequent Shuttles array to return lag times. Where lagtime of $k^{th}$ shuttle is $lagtime_k = (s_k - (delay_k\:mod\;s_k))\:mod\;s_k$


In [4]:
subsequentShuttles = subsequentShuttles.map(s => {
    return {
        ...s,
        lag_time: (s.shuttle - (s.delay % s.shuttle)) % s.shuttle
    }
});

subsequentShuttles

[
  { shuttle: 19, delay: 0, lag_time: 0 },
  { shuttle: 41, delay: 9, lag_time: 32 },
  { shuttle: 37, delay: 13, lag_time: 24 },
  { shuttle: 787, delay: 19, lag_time: 768 },
  { shuttle: 13, delay: 32, lag_time: 7 },
  { shuttle: 23, delay: 42, lag_time: 4 },
  { shuttle: 29, delay: 48, lag_time: 10 },
  { shuttle: 571, delay: 50, lag_time: 521 },
  { shuttle: 17, delay: 67, lag_time: 1 }
]

In [5]:
console.log(`The answer "t" is the solution to the simultaneous congrugencies : \n${subsequentShuttles.map(a => `t ≡ ${a.lag_time} (mod ${a.shuttle})`).join("\n")}`);

The answer "t" is the solution to the simultaneous congrugencies : 
t ≡ 0 (mod 19)
t ≡ 32 (mod 41)
t ≡ 24 (mod 37)
t ≡ 768 (mod 787)
t ≡ 7 (mod 13)
t ≡ 4 (mod 23)
t ≡ 10 (mod 29)
t ≡ 521 (mod 571)
t ≡ 1 (mod 17)


The solution to the simultaneous congrugencies can be found using the [Chinese Reminder theorem](https://en.wikipedia.org/wiki/Chinese_remainder_theorem).
The search for the first $t$ that satisfies the above congrugencies is done here using [sieving](https://en.wikipedia.org/wiki/Chinese_remainder_theorem#Search_by_sieving).

Alternatively this can be solved using [Extended Euclidean algorithm](https://en.wikipedia.org/wiki/Extended_Euclidean_algorithm). An implementation of which is in the [Julia solution](./jl-13.jl)

In [6]:
/** Helper Function ported from AoC19 (https://github.com/adhokshaja/AdventOfCode2019/blob/master/MathHelpers.js), slightly refactored
 * Finds the Greatest Common Divisor of the given array of numbers
 * @param {[Number]} param0 Input array
 */
function GCD (n1, n2, ...others){
    if(!n1 || !n2) {
      return n1||n2;
    }
    let x = Math.abs(n1),
        y = Math.abs(n2);
    while (y) {
        let temp = y;
        y = x % y;
        x = temp;
    } 
    if(others.length<=0){
        return x;
    }

    return GCD(x,...others);  
}


/**
 * Chinese Reminder Theorem - Solve system of Congruent Equations using  
 * @param {[{mod:Number,rem:number}]} eqns Representation of the system of congrugencies
 */
function solveSimultaneousCongrugencies(eqns){

  const eqns_sorted = eqns.sort((a,b)=> b.mod - a.mod);

  // Solution using Sieving (https://en.wikipedia.org/wiki/Chinese_remainder_theorem#Search_by_sieving)

  let acc = eqns_sorted[0].rem;
  let currMultiple = 1;

  for (let i = 1; i < eqns_sorted.length; i++) {
    const { mod: prevMod } = eqns_sorted[i - 1];
    currMultiple = prevMod * currMultiple / GCD(prevMod, currMultiple);

    const { mod, rem } = eqns_sorted[i];
   
    while (acc % mod != rem) {
      acc += currMultiple;
    }

  }

  return acc;
}

The `solveSimultaneousCongrugencies` function expects an array of the form `[ { mod, rem } ]`, where mod is the modulii and rem is the reminders. In this case `mod = shuttle`, `rem = lag_time`

In [7]:
var congrugencyRepresentation = subsequentShuttles.map( a=> {return {mod:a.shuttle, rem:a.lag_time}});
congrugencyRepresentation

[
  { mod: 19, rem: 0 },
  { mod: 41, rem: 32 },
  { mod: 37, rem: 24 },
  { mod: 787, rem: 768 },
  { mod: 13, rem: 7 },
  { mod: 23, rem: 4 },
  { mod: 29, rem: 10 },
  { mod: 571, rem: 521 },
  { mod: 17, rem: 1 }
]

In [8]:
solveSimultaneousCongrugencies(congrugencyRepresentation)

500033211739354